## SF crime data analysis and modeling

In [2]:
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from ggplot import *
import warnings

import os
os.environ["PYSPARK_PYTHON"] = "python3"

In [3]:
'''
import urllib.request
urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))
'''

Out[ 3 ]: '\nimport urllib.request\nurllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")\ndbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv")\ndisplay(dbutils.fs.ls("dbfs:/laioffer/spark_hw1/data/"))\n'

In [4]:
data_path = "dbfs:/laioffer/spark_hw1/data/sf_03_18.csv"
# use this file name later

In [5]:
# read data from the data storage
# please upload your data into databricks community at first. 
crime_data_lines = sc.textFile(data_path)
#prepare data 
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the first line of data
#display(crimes.take(3))

#get the total number of data 
print(crimes.count())


['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId', 'SF Find Neighborhoods', 'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods', ':@computed_region_yftq_j783', ':@computed_region_p5aj_wyqh', ':@computed_region_rxqg_mtj9', ':@computed_region_bh8s_q3mv', ':@computed_region_fyvs_ahh9', ':@computed_region_9dfj_4gjx', ':@computed_region_n4xg_c4py', ':@computed_region_4isq_27mq', ':@computed_region_fcz8_est8', ':@computed_region_pigm_ib2e', ':@computed_region_9jxd_iqea', ':@computed_region_6pnf_4xz7', ':@computed_region_6ezc_tdp2', ':@computed_region_h4ep_8xdi', ':@computed_region_nqbw_i6c3', ':@computed_region_2dwj_jsy4']
2215024

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("crime analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)
#display(df_opt1)
df_opt1.createOrReplaceTempView("sf_crime")

In [7]:
crimeCategory=spark.sql("SELECT  category, COUNT(*) AS Count FROM sf_crime GROUP BY category ORDER BY Count DESC").show()

+--------------------+------+
 category| Count|
+--------------------+------+
 LARCENY/THEFT|480448|
 OTHER OFFENSES|309358|
 NON-CRIMINAL|238323|
 ASSAULT|194694|
 VEHICLE THEFT|126602|
 DRUG/NARCOTIC|119628|
 VANDALISM|116059|
 WARRANTS|101379|
 BURGLARY| 91543|
 SUSPICIOUS OCC| 80444|
 MISSING PERSON| 64961|
 ROBBERY| 55867|
 FRAUD| 41542|
 SECONDARY CODES| 25831|
FORGERY/COUNTERFE...| 23050|
 WEAPON LAWS| 22234|
 TRESPASS| 19449|
 PROSTITUTION| 16701|
 STOLEN PROPERTY| 11891|
SEX OFFENSES, FOR...| 11742|
+--------------------+------+
only showing top 20 rows

In [8]:
spark.sql("SELECT PdDistrict,count(*) AS number FROM sf_crime GROUP BY PdDistrict ORDER BY number DESC").show()

+----------+------+
PdDistrict|number|
+----------+------+
 SOUTHERN|399785|
 MISSION|300076|
 NORTHERN|272713|
 CENTRAL|226255|
 BAYVIEW|221000|
 INGLESIDE|194180|
TENDERLOIN|191746|
 TARAVAL|166971|
 PARK|125479|
 RICHMOND|116818|
 null| 1|
+----------+------+

In [9]:
spark.sql("SELECT COUNT(*)as number FROM sf_crime WHERE DayOfWEEK='Sunday' and Y>'37.4716788' and Y<'37.773972' and X>'-122.2426773' and X<'-122.431297'").show()

+------+
number|
+------+
 87694|
+------+

In [10]:
spark.sql("SELECT year(date)as year,month(date) as month, count(*)as number FROM (SELECT TO_DATE(CAST(UNIX_TIMESTAMP(Date, 'MM/dd/yyyy') AS TIMESTAMP))as date FROM sf_crime)tb GROUP BY year(date),month(date) HAVING year between 2015 and 2018 ORDER BY year, month").show()

+----+-----+------+
year|month|number|
+----+-----+------+
2015| 1| 13606|
2015| 2| 12329|
2015| 3| 13929|
2015| 4| 12959|
2015| 5| 13729|
2015| 6| 13304|
2015| 7| 13365|
2015| 8| 13730|
2015| 9| 12896|
2015| 10| 13147|
2015| 11| 12091|
2015| 12| 11441|
2016| 1| 12967|
2016| 2| 12106|
2016| 3| 12380|
2016| 4| 12328|
2016| 5| 12732|
2016| 6| 12094|
2016| 7| 12191|
2016| 8| 12471|
+----+-----+------+
only showing top 20 rows

The crime number is stable during these years. But it is relatively low in Dec 2015 duirng Christmas month. Besides, it is surprisingly low between Feburary and May in 2018. I guess one law regulation which limited crime rate may be introduced during this time.

In [12]:
spark.sql("Select to_timestamp((CAST(UNIX_TIMESTAMP(new_time, 'MM/dd/yyyy HH:mm') AS TIMESTAMP))) as time FROM (SELECT CONCAT(Date,' ',Time )as new_time FROM sf_crime)tb ").show()

+-------------------+
 time|
+-------------------+
2018-05-15 10:30:00|
2018-05-15 04:14:00|
2018-05-15 02:01:00|
2018-05-15 02:01:00|
2018-05-15 02:01:00|
2018-05-15 01:27:00|
2018-05-15 01:25:00|
2018-05-15 01:25:00|
2018-05-15 00:19:00|
2018-05-15 00:19:00|
2018-05-14 23:43:00|
2018-05-14 23:43:00|
2018-05-14 21:45:00|
2018-05-14 21:30:00|
2018-05-14 21:30:00|
2018-05-14 20:42:00|
2018-05-14 20:02:00|
2018-05-14 20:01:00|
2018-05-14 19:47:00|
2018-05-14 19:47:00|
+-------------------+
only showing top 20 rows

In [13]:
crime_num=spark.sql("SELECT hour(time) as hour,count(*)as count FROM(Select to_timestamp((CAST(UNIX_TIMESTAMP(new_time, 'MM/dd/yyyy HH:mm') AS TIMESTAMP))) as time FROM (SELECT CONCAT(Date,' ',Time )as new_time FROM sf_crime)tb)tb2 WHERE year(time)==2016 AND month(time)==12 AND day(time)==15 GROUP BY hour(time) ORDER BY(hour(time)) ")
display(crime_num)

hour,count
0,22
1,10
2,12
3,9
4,1
5,6
6,5
7,3
8,24
9,23


We can see from the figure of Dec 15 2018 above. The crime count is relative higher during day time compared to midnight. It must be mentioned that it is especiallay dangereous from 18pm to 20pm. So the visitors are advised to be carefull during this period.

In [15]:
spark.sql("SELECT PdDistrict,count(*)as number FROM sf_crime GROUP BY PdDistrict ORDER BY number DESC LIMIT 3").show()

+----------+------+
PdDistrict|number|
+----------+------+
 SOUTHERN|399785|
 MISSION|300076|
 NORTHERN|272713|
+----------+------+

In [16]:
category3num=spark.sql("SELECT PdDistrict,category,hour(time) as hour,count(*) as number FROM(Select PdDistrict,category,to_timestamp((CAST(UNIX_TIMESTAMP(new_time, 'MM/dd/yyyy HH:mm') AS TIMESTAMP))) as time FROM (SELECT PdDistrict,category,CONCAT(Date,' ',Time )as new_time FROM sf_crime)tb)tb1 WHERE PdDistrict=='SOUTHERN' or PdDistrict=='MISSION' or PdDistrict=='NORTHERN'  GROUP BY PdDistrict,category,hour(time) ")
display(category3num)

PdDistrict,category,hour,number
SOUTHERN,ASSAULT,8,1059
SOUTHERN,ASSAULT,11,1387
NORTHERN,SUICIDE,17,11
MISSION,BRIBERY,21,12
MISSION,BRIBERY,1,5
NORTHERN,MISSING PERSON,4,31
SOUTHERN,DRIVING UNDER THE INFLUENCE,4,12
NORTHERN,BRIBERY,16,3
SOUTHERN,LIQUOR LAWS,22,40
NORTHERN,EXTORTION,21,5


According to the graph, LARCENY/THEFT are more likely to happen among the 3 district. And we should also aviod assault to happen.

In [18]:
categortinhour=spark.sql("SELECT PdDistrict,hour(time)as hour,count(*) as number FROM(Select PdDistrict,category,to_timestamp((CAST(UNIX_TIMESTAMP(new_time, 'MM/dd/yyyy HH:mm') AS TIMESTAMP))) as time FROM (SELECT PdDistrict,category,CONCAT(Date,' ',Time )as new_time FROM sf_crime)tb)tb1 WHERE PdDistrict=='SOUTHERN' or PdDistrict=='MISSION' or PdDistrict=='NORTHERN'  GROUP BY PdDistrict,hour Order by hour ASC")
display(categortinhour)

PdDistrict,hour,number
NORTHERN,0,14472
SOUTHERN,0,20171
MISSION,0,16797
SOUTHERN,1,11203
MISSION,1,11125
NORTHERN,1,9291
MISSION,2,8930
NORTHERN,2,7809
SOUTHERN,2,9295
MISSION,3,5599


We can get from the graph above, the police should be assign more from 12pm to 12am, especially in Southern District.

In [20]:
spark.sql("SELECT category,(ROUND(SUM(CASE WHEN resolution=='UNFOUNDED' THEN 1 ELSE 0 END)*100/count(*),2))as percentage FROM sf_crime GROUP BY category ").show()

+--------------------+----------+
 category|percentage|
+--------------------+----------+
 FRAUD| 0.69|
 SUICIDE| 0.7|
 LIQUOR LAWS| 0.07|
 SECONDARY CODES| 0.96|
 FAMILY OFFENSES| 3.63|
 MISSING PERSON| 1.56|
 OTHER OFFENSES| 0.56|
DRIVING UNDER THE...| 0.11|
 WARRANTS| 0.13|
 ARSON| 0.64|
FORGERY/COUNTERFE...| 1.7|
 GAMBLING| 0.57|
 BRIBERY| 0.62|
 ASSAULT| 0.65|
 DRUNKENNESS| 0.28|
 EXTORTION| 1.08|
 TREA| 0.0|
 WEAPON LAWS| 0.7|
 LOITERING| 0.12|
 SUSPICIOUS OCC| 3.2|
+--------------------+----------+
only showing top 20 rows

We can see from above. The categories of missing person,family offenses, non-criminal or etc have relatively higher percentage in resolution. So the police department should pay more attention on these crime.

#Conclusion

I did some basic sql query to analyse the crime number of San Francisco, regarding category count, resolution percentage and crime number among hours in different districts or etc in Spark SQL.
And Theft was more likely to happen among all crime. Southern, Northern and Mission are 3 most dangerous districts in SF. The police should assign more resource to these area and several high resolution percentage crime type.